In [2]:
import pickle
import numpy as np
path = "D:\\work\\Research\\COVID_wiki\\repo files\\ambiguous_data"
import pandas as pd
import csv
a = pd.read_csv(path+"\\train.txt",sep = " ",quoting=csv.QUOTE_NONE,header = None,skip_blank_lines = False).fillna("-@@-")
_a = a
a = np.array(a)

In [3]:
label_list = a.T[1]
input_list = a.T[0]

In [4]:
def get_span(seq):
        i = 0
        l = seq[0]
        while l == seq[i]:
            i+=1
            if seq[i]=="-@@-":
                break

        return l,i


In [5]:
inp_spans = []
label_spans = []
i =0
s = label_list
sp = input_list
while i < len(s):
    label = s[i]
  
    inp = sp[i]
    
    if label!="O" and label!="-@@-":
        
        span,span_len = get_span(s[i:])
        label_span =s[i:i+span_len]
        inp_span =  sp[i:i+span_len]
        inp_spans.append(inp_span)
        label_spans.append(label_span)
        i+=span_len
    i+=1 
inp_list_spans = inp_spans
inp_spans = [" ".join(inp_span) for inp_span in inp_spans]  
label_spans = [l[0] for l in label_spans]      
span_length = {span:len(list_span) for span,list_span in zip(inp_spans,inp_list_spans)}        

In [6]:
span_label_counts = {span:{} for span in inp_spans}

for span, label in zip(inp_spans,label_spans):
    if span not in span_label_counts:
        span_label_counts[span] = {label:1}
    else:
        if label not in span_label_counts[span]:
            span_label_counts[span][label]=1
        else:
            span_label_counts[span][label]+=1

In [7]:
span_uncertain = {}
span_majority_labels = {}

for span, labels in span_label_counts.items():
    
    total = np.sum(list(labels.values()))
    m=  np.max(list(labels.values()))
    ind  = np.argmax(list(labels.values()))
    l = max(labels,key = labels.get)
    if m/total >0.9:
        span_uncertain[span] = False
        span_majority_labels[span] = l

In [8]:
max_span_length = np.max([len(span) for span in span_uncertain])


In [9]:
#a = pd.read_csv(path+"\\test.txt",sep = " ",quoting=csv.QUOTE_NONE,header = None,skip_blank_lines = False).fillna("-@@-")
f = open(path+"\\test.txt","r",encoding = "utf8")
a = f.readlines()
test = []
sent = []
for line in a:
    if len(line) == 1:
        
        test.append(np.array(sent, np.object))
        sent = []
    else:
        a,b = line.split(" ")
        b = b.strip("\n")
        sent.append(np.array([a,b],np.object))
f.close()
test= np.array(test,np.object)

In [10]:
input_sentences = [t.T[0] for t in test]
labels = [t.T[1] for t in test]


In [11]:
k = 10
def label_sentence(s):
    labels = ["O"]* len(s)
    query_str = " ".join(s)
    query = s
    label_changes = []
    for span in span_majority_labels:
        if " "+span+" "  in query_str:
            ind = query_str.find(span)
            c = query_str[:ind].count(" ")
            label_changes.append([span_majority_labels[span], c,c+span_length[span],span_length[span]])
            #print(span, span_majority_labels[span], query[c:c+span_length[span]])
            

    label_changes = sorted(label_changes, key = lambda x: x[3])

    for change in label_changes:
        for i in range(change[1],change[2]):
            labels[i] = change[0]
    return query, labels


In [12]:
import tqdm
preds = []
for d in tqdm.tqdm(input_sentences):
    query, pred = label_sentence(d)
    preds.append(pred)

100%|██████████| 15590/15590 [00:34<00:00, 452.65it/s]


In [19]:

def end_to_end_metrics(preds_list,out_label_list, weak = True):
    
    def get_span(seq):
        i = 0
        l = seq[0]
        while l == seq[i]:
            i+=1
            if i == len(seq):
                break

        return l,i
    def compare_spans(label_span,pred_span,weak = True):
        if weak:
            t = label_span[0]
            if t in pred_span:
                return True
            else:
                return False
        else:
            for i,t in enumerate(label_span):
                if t!=pred_span[i]:
                    return False
            return True

    TP = 0
    TN = 0
    FP = 0
    FN = 0

    for j,s in enumerate(out_label_list):
        sp = preds_list[j]
        i = 0
        while i < len(s):
        #for i in range(len(s)):
            label = s[i]
            pred = sp[i]
            if label =="O":
                if pred == "O":
                    TN+=1
                else:
                    FP+=1

            if label!="O":
                span,span_len = get_span(s[i:])
                label_span =s[i:i+span_len]
                pred_span =  sp[i:i+span_len]

                if compare_spans(label_span,pred_span, weak = weak):
                    TP+=1
                else:
                    FN+=1
                    

                i+=span_len
                
            i+=1
    if TP+FP ==0:
        precision = 0
    else:
        precision = TP/(TP+FP)
    
    if TP+FN ==0:
        recall = 0
    else:
        recall = TP/(TP+FN)
    
    if precision + recall!=0:
        F1 = 2*(precision*recall)/(precision+recall)
    else:
        F1 = 0

    dec= "Weak" if weak else "Strong"
    metrics = {dec+" Precision":precision,dec+" Recall":recall,dec+" F1":F1}
    return metrics

In [23]:
end_to_end_metrics(preds,labels,weak = True)


{'Weak Precision': 0.31685555280174843,
 'Weak Recall': 0.7955085416902364,
 'Weak F1': 0.4531992716843106}

In [24]:
end_to_end_metrics(preds,labels,weak = False)

{'Strong Precision': 0.31339024887338934,
 'Strong Recall': 0.7828374250480824,
 'Strong F1': 0.44759609942267575}